In [0]:
# CHECK EXISTING DATABRICKS CONFIGURATION
print("=" * 80)
print("DATABRICKS ENVIRONMENT CHECK")
print("=" * 80)

# 1. Check Secret Scopes
print("\n1. CHECKING SECRET SCOPES")
print("-" * 80)
try:
    scopes = dbutils.secrets.listScopes()
    if len(scopes) > 0:
        print(f"Found {len(scopes)} secret scope(s):")
        for scope in scopes:
            print(f"  - {scope.name} (Backend: {scope.backendType})")
        
        # List secrets in each scope
        print("\n   Secrets in each scope:")
        for scope in scopes:
            try:
                secrets = dbutils.secrets.list(scope.name)
                print(f"\n   Scope: {scope.name}")
                if len(secrets) > 0:
                    for secret in secrets:
                        print(f"     - {secret.key}")
                else:
                    print("     (no secrets)")
            except Exception as e:
                print(f"     Error listing secrets: {e}")
    else:
        print("No secret scopes found")
except Exception as e:
    print(f"Error checking scopes: {e}")

# 2. Check Existing Mounts
print("\n2. CHECKING EXISTING MOUNTS")
print("-" * 80)
try:
    mounts = dbutils.fs.mounts()
    if len(mounts) > 0:
        print(f"Found {len(mounts)} mount(s):")
        for mount in mounts:
            print(f"  - {mount.mountPoint}")
            print(f"    Source: {mount.source}")
            
            # Check if it's our storage account
            if "dataservicesdlsdev" in mount.source:
                print(f"    *** THIS IS YOUR STORAGE ACCOUNT! ***")
    else:
        print("No mounts found")
except Exception as e:
    print(f"Error checking mounts: {e}")

# 3. Check if dataservicesdlsdev is mounted
print("\n3. CHECKING FOR dataservicesdlsdev MOUNT")
print("-" * 80)
possible_mount_points = [
    "/mnt/dataservicesdlsdev",
    "/mnt/data",
    "/mnt/bronze",
    "/mnt/raw",
    "/mnt/landing"
]

for mount_point in possible_mount_points:
    try:
        files = dbutils.fs.ls(mount_point)
        print(f"✓ {mount_point} EXISTS!")
        print(f"  Found {len(files)} items")
        
        # Check for mg-gold-raw-files
        for item in files:
            if "mg-gold-raw-files" in item.name:
                print(f"  ✓✓✓ FOUND mg-gold-raw-files at {mount_point}/{item.name}")
    except:
        print(f"✗ {mount_point} does not exist")

# 4. Try to access mg-gold-raw-files directly
print("\n4. TRYING DIRECT ACCESS TO mg-gold-raw-files")
print("-" * 80)
direct_paths = [
    "/mnt/dataservicesdlsdev/mg-gold-raw-files",
    "/mnt/data/mg-gold-raw-files",
    "/mnt/bronze/mg-gold-raw-files"
]

for path in direct_paths:
    try:
        files = dbutils.fs.ls(path)
        print(f"✓ SUCCESS! {path} is accessible")
        print(f"  Found {len(files)} files:")
        for f in files[:5]:  # Show first 5
            print(f"    - {f.name}")
        if len(files) > 5:
            print(f"    ... and {len(files)-5} more")
        break
    except Exception as e:
        print(f"✗ {path} not accessible")

print("\n" + "=" * 80)
print("ENVIRONMENT CHECK COMPLETE")
print("=" * 80)

DATABRICKS ENVIRONMENT CHECK

1. CHECKING SECRET SCOPES
--------------------------------------------------------------------------------
Found 2 secret scope(s):
Error checking scopes: 'SecretScope' object has no attribute 'backendType'

2. CHECKING EXISTING MOUNTS
--------------------------------------------------------------------------------
Found 13 mount(s):
  - /databricks-datasets
    Source: databricks-datasets
  - /Volumes
    Source: UnityCatalogVolumes
  - /mnt/dlsvnet
    Source: abfss://tiservicedata@dataservicesdlsdev.dfs.core.windows.net/
    *** THIS IS YOUR STORAGE ACCOUNT! ***
  - /databricks/mlflow-tracking
    Source: databricks/mlflow-tracking
  - /databricks-results
    Source: databricks-results
  - /databricks/mlflow-registry
    Source: databricks/mlflow-registry
  - /mnt/swanleap-prod
    Source: s3a://swanleap-prod-dynamo-exports-us-east-1-128237996359
  - /mnt/dls_cleodata
    Source: abfss://cleodata-edi@dataservicesdlsdev.dfs.core.windows.net/
    *** THIS

In [0]:
# FIND YOUR STORAGE ACCESS KEY IN EXISTING SECRET SCOPES

print("Searching for storage access key in your 2 secret scopes...")
print("=" * 80)

# Get list of scopes
scopes = dbutils.secrets.listScopes()

for scope in scopes:
    scope_name = scope.name
    print(f"\nScope: {scope_name}")
    print("-" * 80)
    
    try:
        # List all secrets in this scope
        secrets = dbutils.secrets.list(scope_name)
        
        print(f"Found {len(secrets)} secrets:")
        for secret in secrets:
            print(f"  - {secret.key}")
            
            # Check if this looks like a storage key
            if any(keyword in secret.key.lower() for keyword in ['storage', 'dls', 'dataservices', 'adls', 'key', 'account']):
                print(f"    ^ This might be your storage key!")
                
    except Exception as e:
        print(f"  Error listing secrets: {e}")

print("\n" + "=" * 80)
print("Which scope and secret name should we use for the mount?")

Searching for storage access key in your 2 secret scopes...

Scope: DataServices-KeyVault-Scope
--------------------------------------------------------------------------------
Found 199 secrets:
  - ADLSConnectionKey
    ^ This might be your storage key!
  - ADLSConnectionKey02
    ^ This might be your storage key!
  - ADLSConnectionKeyVNet
    ^ This might be your storage key!
  - ADLSConnectionString
    ^ This might be your storage key!
  - APIPasswordTest
  - Azure-sql-etl-legacy
  - AzureDatabricks-Token
  - AzureServiceBusDevSAS
  - AzureSqlConnectionString
  - AzureSqlDatabase
  - AzureSqlDatabaseProd
  - AzureSqlLegacyConnectionString
  - AzureSqlPassword
  - AzureSqlPasswordProd
  - AzureSqlServerName
  - AzureSqlServerNameProd
  - AzureSqlUsername
  - AzureSqlUsernameProd
  - BatchAccountAccessKey
    ^ This might be your storage key!
  - Bitbucket-AzureDevops-user-app-password
  - Bitbucket-AzureDevops-user-app-username
  - BlobStorageConnectionString
    ^ This might be yo

In [0]:
# MOUNT mg-gold-raw-files CONTAINER (FIXED)
# Using wasbs:// scheme instead of abfss://

storage_account = "dataservicesdlsdev"
container = "mg-gold-raw-files"
mount_point = "/mnt/mg-gold-raw-files"

print("=" * 80)
print("MOUNTING mg-gold-raw-files")
print("=" * 80)
print(f"Storage Account: {storage_account}")
print(f"Container: {container}")
print(f"Mount Point: {mount_point}")

# Check if already mounted
try:
    dbutils.fs.ls(mount_point)
    print(f"\n✓ Already mounted at {mount_point}")
    
    # Show contents
    files = dbutils.fs.ls(mount_point)
    print(f"Found {len(files)} items")
    if len(files) > 0:
        display(dbutils.fs.ls(mount_point))
    
except:
    print(f"\nCreating new mount...")
    
    # Get access key from existing secret scope
    access_key = dbutils.secrets.get("DataServices-KeyVault-Scope", "ADLSConnectionKey")
    
    configs = {
        f"fs.azure.account.key.{storage_account}.blob.core.windows.net": access_key
    }
    
    try:
        dbutils.fs.mount(
            source=f"wasbs://{container}@{storage_account}.blob.core.windows.net/",
            mount_point=mount_point,
            extra_configs=configs
        )
        print(f"\n✓ Successfully mounted {mount_point}")
        
        # Verify
        files = dbutils.fs.ls(mount_point)
        print(f"✓ Verified - found {len(files)} items")
        
        if len(files) > 0:
            print("\nContents:")
            display(dbutils.fs.ls(mount_point))
        else:
            print("\nContainer is empty - upload MercuryGate files to begin")
            
    except Exception as e:
        print(f"\n✗ Error: {e}")
        print("\nTroubleshooting:")
        print("  - Try ADLSConnectionKeyVNet secret instead")
        print("  - Verify container 'mg-gold-raw-files' exists in Azure Portal")

print("\n" + "=" * 80)
print("MOUNT COMPLETE")
print("=" * 80)
print(f"\nYour container is now accessible at: {mount_point}")

MOUNTING mg-gold-raw-files
Storage Account: dataservicesdlsdev
Container: mg-gold-raw-files
Mount Point: /mnt/mg-gold-raw-files

Creating new mount...

✓ Successfully mounted /mnt/mg-gold-raw-files
✓ Verified - found 0 items

Container is empty - upload MercuryGate files to begin

MOUNT COMPLETE

Your container is now accessible at: /mnt/mg-gold-raw-files
